# Programmation Fonctionnelle en Python
(Introduction)

La programmation fonctionnelle est un paradigme de programmation de type déclaratif qui considère le calcul en tant qu'évaluation de fonctions mathématiques. ([wiki](https://fr.wikipedia.org/wiki/Programmation_fonctionnelle))


$$ f: x \mapsto f(x) $$

Le respect du formalisme mathématique impose certaines rêgles mais permet de plus facilement s'approprier les propriétés des objets mathématiques utilisés.

Parmi les règles à respecter :
* l'immutabilité
* l'absence d'effets de bords (fonctions pures)
* les fonctions sont des objets comme les autres

## Immutabilité

En mathématiques, les _variables_ sont en fait des _valeurs_. Une fois définie une égalité entre une "variable" `x` et une valeur (par exemple, 2), on ne revient jamais dessus.

In [1]:
x = 2 # x EST 2, c'est la même chose

In [2]:
x = 3 # N'a pas de sens en programmation fonctionnelle, vu qu'on a dit que x = 2

En plus, c'est pratique !

### Mutable

In [3]:
import pandas as pd
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])
df

,a,b
0,0,1
1,2,3


In [5]:
df.drop('b', axis=1, inplace=True)  # Can't be run twice

ValueError: labels ['b'] not contained in axis

### Immutable

In [6]:
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])

In [8]:
df2 = df.drop('b', axis=1)  # Run it as many times as you wish !

## Fonctions "Pures"

Une fonction pure est une fonction dont le résultat ne dépend que de ses arguments, et qui n'a pas d'effet de bord.

In [9]:
def impure_add_columns():
    df['c'] = df['a'] + df['b']
    return df

In [10]:
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])
impure_add_columns()

,a,b,c
0,0,1,1
1,2,3,5


In [11]:
df = pd.DataFrame([[0, 1], [2, 3], [4, 5]], columns = ['a', 'b'])
impure_add_columns()

,a,b,c
0,0,1,1
1,2,3,5
2,4,5,9


Au sens mathématique, ce n'est même pas une fonction...<br>
=> Danger ! Je pense que ma fonction renvoie toujours la même chose, mais si je change une valeur quelque part dans mon code, la valeur de sortie change aussi !

In [12]:
def another_impure_add_columns(df):
    df['c'] = df['a'] + df['b']

In [13]:
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])
another_impure_add_columns(df)
df

,a,b,c
0,0,1,1
1,2,3,5


Violation de l'immutabilité ! => Difficulté à paralléliser.<br>
Violation de l'absence d'effets de bord ! => Difficulté à anticiper les effets d'une fonction.

In [14]:
def pure_add_column(df):
    return df.assign(c = df['a'] + df['b'])

In [15]:
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])
pure_add_column(df)

,a,b,c
0,0,1,1
1,2,3,5


In [16]:
df

,a,b
0,0,1
1,2,3


## Fonctions Anonymes

Parfois, on s'en tape un peu de donner un nom à une fonction, on veut juste savoir ce qu'elle fait :

In [17]:
lambda x, y : x + y

<function __main__.<lambda>>

In [18]:
df.apply(lambda x: x.sum())

a    2
b    4
dtype: int64

## Fonctions d'ordre supérieur

Parce que passer une fonction à une autre fonction, c'est puissant

### Un logger paramétrique ;-)

In [23]:
def print_logger(message):
    print(message)
    
def file_logger(message):
    with open('output.log', 'a') as log:
        log.write(message + "\n")

def log(message, logger):
    logger(message)
    
def reset():
    import os
    try:
        os.remove('output.log')
    except FileNotFoundError:
        pass

In [20]:
log("Hello World", print_logger)

Hello World


In [24]:
reset()
log("Hello World", file_logger)
log("This is an awesome log", file_logger)

In [25]:
!cat output.log

Hello World
This is an awesome log


Au fait !! Un décorateur est une fonction d'ordre supérieur !

### Les décorateurs

In [26]:
def assert_square(decorated_function):
    def new_function(df):
        n, m = df.shape
        assert n == m, "Not a square DataFrame"
        return decorated_function(df)
    return new_function

In [27]:
@assert_square
def decorated_function(df):
    print("SUCCESS")

In [28]:
df = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'b'])
df2 = pure_add_column(df)

decorated_function(df2)

AssertionError: Not a square DataFrame

Strictly equal to :

In [29]:
def non_decorated_function(df):
    print("SUCCESS")

decorated_function2 = assert_square(non_decorated_function)

decorated_function2(df2)

AssertionError: Not a square DataFrame

### Plus de décorateurs !!!

In [30]:
from functools import wraps

def assert_columns(columns):
    """
    Une fonction qui renvoie un décorateur
    """
    
    def decorator(f):
        """
        Un superbe décorateur
        """
        
        @wraps(f)
        def run(df):
            for col in columns:
                assert col in df.columns, "column {} must be present !".format(col)
            return f(df)
        
        return run
    
    return decorator

In [31]:
@assert_columns(['a', 'b'])
def add_column(df):
    "Create columns 'c' by summing 'a' and 'b'"
    return df.assign(c = df['a'] + df['b'])

In [32]:
df3 = pd.DataFrame([[0, 1], [2, 3]], columns = ['a', 'd'])

add_column(df3)

AssertionError: column b must be present !

In [33]:
add_column.__doc__

"Create columns 'c' by summing 'a' and 'b'"

## Lazy evaluation

= la capacité de n'évaluer une expression que lorsqu'on en a besoin

In [34]:
def infinite_range():
    """
    I can return as many element as you wish !
    """
    i = 0
    while True:
        yield i
        i += 1

In [35]:
for i in infinite_range():
    print(i)
    if i >= 10:
        break

0
1
2
3
4
5
6
7
8
9
10


Concrêtement, on a ici un moyen de définir des ensembles infinis.

$$ \mathbb{N}, \mathbb{Z}, \mathbb{Q}... $$

Python ne gère pas bien la _lazy evaluation_ en dehors des générateurs:

In [36]:
import time

def one():
    time.sleep(1)
    return 1

%time x = [one(), one(), one()][0]

CPU times: user 579 µs, sys: 938 µs, total: 1.52 ms
Wall time: 3 s


Même si seul le premier élément doit être évalué, les trois éléments de la liste sont en fait pré-calculés

## Fonctionnel / Objet

Les deux concepts ne sont pas exclusifs !!

In [37]:
class Invoice(object):
    def __init__(self, customer, amount):
        self.amount = amount
        self.customer = customer
        
    def change_amount(self, amount):
        return Invoice(self.customer, amount)

In [38]:
invoice = Invoice("VDU&Co", 1000)

# Erreur pendant la facturation
new_invoice = invoice.change_amount(100)

Immutabilité, absence d'effets de bords: cet objet respecte le paradigme fonctionnel !

# Merci !

### Pour aller plus loin :
* [Article wikipédia de la programmation Fonctionnelle (en français)](https://fr.wikipedia.org/wiki/Programmation_fonctionnelle)
* [Ebook sur la Programmation Fonctionnelle (en JS)](https://github.com/MostlyAdequate/mostly-adequate-guide)
* [Ebook sur la Programmation Fonctionnelle en Python (en Python donc)](http://www.oreilly.com/programming/free/functional-programming-python.csp)
* [Functor, Applicative & Monads (en Haskell, avec des dessins)](http://adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html)

## Bonus : Monads 

Les Monades sont : 
- des containeurs
- capable d'enchainer des fonctions renvoyant des monades

Exemple : le Maybe Monade

In [39]:
class Maybe(object):
    def __init__(self):
        pass

class Just(Maybe):
    def __init__(self, value):
        self.value = value
        
    def bind(self, f):
        return f(self.value)
    
    def __repr__(self):
        return "Just({})".format(self.value)
        
class Nothing(Maybe):
    def __init__(self):
        pass
    
    def bind(self, f):
        return self
    
    def __repr__(self):
        return "Nothing"

In [40]:
def divide_by_zero(x):
    return Nothing()

def divide_by_2(x):
    return Just(x / 2)

In [41]:
Just(8).bind(divide_by_2)

Just(4.0)

In [42]:
Just(8).bind(divide_by_zero)

Nothing

In [43]:
Nothing().bind(divide_by_2)

Nothing

In [44]:
Nothing().bind(divide_by_zero)

Nothing

In [45]:
Just(8).bind(divide_by_2).bind(divide_by_2)

Just(2.0)

In [46]:
Just(8).bind(divide_by_zero).bind(divide_by_2).bind(divide_by_2)

Nothing

### Sans la Maybe Monade

In [47]:
def divide_by_zero(x):
    return None

def divide_by_2(x):
    if x is None:
        return x / 2
    else:
        return None

La logique de vérification de la valeur nulle doit être incorporée dans le code des fonctions de transformations.

# This is it !